<a href="https://colab.research.google.com/github/Parry1J/AIBootCamp/blob/main/Hugging_Face_Example_implementaion_with_Microservice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install transformers datasets

In [10]:
from transformers import pipeline
task = "sentiment-analysis"

classifier = pipeline(task)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [11]:
from datasets import load_dataset
dataset = load_dataset("imdb")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [12]:
import numpy as np

# Define the range
start = 0
end = len(dataset["train"])
num_indices = 100

# Create a range of values and then randomly select from them
indices_range = np.arange(start, end)
random_indices = np.random.choice(indices_range, size=num_indices, replace=False)


In [13]:
classifier(dataset["train"][0]["text"]), dataset["train"][0]["label"]

([{'label': 'POSITIVE', 'score': 0.7872804403305054}], 0)

In [8]:
hits = 0
misses = 0
for index in random_indices:
  ix = int(index)
  sentiment = classifier(dataset["train"][ix]["text"][:512])
  text_label = 'POSITIVE' if dataset["train"][ix]["label"] == 1 else 'NEGATIVE'
  if sentiment[0]["label"] == text_label:
    hits += 1
  else:
    misses += 1
print(f"Accuracy: {hits / (hits + misses)}")

Accuracy: 0.88


In [14]:
!pip install flask requests

In [19]:
%%writefile app.py
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

task = "sentiment-analysis"
classifier = pipeline(task)

@app.route('/sentiment', methods=['POST'])
def predict():
    data = request.get_json()
    text = data['text']
    sentiment = classifier(text)[0]
    return jsonify(sentiment)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

Overwriting app.py


In [20]:
!nohup python app.py &

nohup: appending output to 'nohup.out'


In [21]:
import requests
response = requests.post('http://localhost:5000/sentiment', json={'text': 'I love this movie!'})

In [22]:
response.json()

{'label': 'POSITIVE', 'score': 0.9998775720596313}